In [14]:
import sys

sys.path.append("/home/kaiwenguo/dev/rnd-ditwo-reference-cond/src")

from datalib import InputPath
from safetensors.torch import load_file

from ditwo.configs.config_dataclasses import WAN_I2V_1_3B_CONFIG
from ditwo.models.transformer.wan import Transformer


config = WAN_I2V_1_3B_CONFIG
model = Transformer(config=config)

resolved_path = InputPath(
    "s3://synthesia-rnd-prd-third-party-models/wan/dit/Wan2.1-I2V-1.3B-multires-bfloat16.safetensors"
).resolve()

state_dict = load_file(resolved_path)

[2025-05-06 15:21:11,752] [INFO] [wan.py:362] 🚀 Attention used: PytorchSDPA 🚀


In [12]:
state_dict.keys()

dict_keys(['final_layer.fc.bias', 'final_layer.fc.weight', 'final_layer.time_modulation_bias', 'patch_embedder.fc.bias', 'patch_embedder.fc.weight', 'text_embedder.0.bias', 'text_embedder.0.weight', 'text_embedder.2.bias', 'text_embedder.2.weight', 'time_embedder.mlp.fc1.bias', 'time_embedder.mlp.fc1.weight', 'time_embedder.mlp.fc2.bias', 'time_embedder.mlp.fc2.weight', 'time_projection.1.bias', 'time_projection.1.weight', 'transformer_blocks.0.cross_attn.fc_k.bias', 'transformer_blocks.0.cross_attn.fc_k.weight', 'transformer_blocks.0.cross_attn.fc_out.bias', 'transformer_blocks.0.cross_attn.fc_out.weight', 'transformer_blocks.0.cross_attn.fc_q.bias', 'transformer_blocks.0.cross_attn.fc_q.weight', 'transformer_blocks.0.cross_attn.fc_v.bias', 'transformer_blocks.0.cross_attn.fc_v.weight', 'transformer_blocks.0.cross_attn.k_norm.weight', 'transformer_blocks.0.cross_attn.q_norm.weight', 'transformer_blocks.0.ffn.0.bias', 'transformer_blocks.0.ffn.0.weight', 'transformer_blocks.0.ffn.2.bia

In [15]:
import numpy as np

new_state_dict = {}
for key, value in state_dict.items():
    if key.startswith("patch_embedder"):
        # Load weights for vision patch embedder
        new_key = key.replace("patch_embedder", "vision_patch_embedder")
        print(new_key, value.shape)
        new_state_dict[new_key] = value
        # Load weights for reference patch embedder
        new_key = key.replace("patch_embedder", "reference_patch_embedder")
        if new_key.endswith("weight"):
            value = value.reshape(value.shape[0], -1, config.in_features)
            new_state_dict[new_key] = value[..., -config.reference_in_features :].flatten(-2)
        elif new_key.endswith("bias"):
            new_state_dict[new_key] = value
        print(new_key, new_state_dict[new_key].shape)
    else:
        new_state_dict[key] = value
model.load_state_dict(new_state_dict, strict=True)

vision_patch_embedder.fc.bias torch.Size([1536])
reference_patch_embedder.fc.bias torch.Size([1536])
vision_patch_embedder.fc.weight torch.Size([1536, 144])
reference_patch_embedder.fc.weight torch.Size([1536, 64])


<All keys matched successfully>